In [1]:
!pip install -q transformers datasets accelerate bitsandbytes google-cloud-storage gguf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from google.colab import drive
from google.cloud import storage
from transformers import AutoModelForCausalLM, AutoTokenizer

# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. Set environment variable for credentials
credentials_path = "/content/drive/MyDrive/gcp-key.json"  # Path to your key file
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credentials_path

# 3. Initialize GCS client
storage_client = storage.Client()
bucket_name = "llm-test-bucket-2025"  # Replace with your bucket name
bucket = storage_client.bucket(bucket_name)

Mounted at /content/drive


In [3]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from google.cloud import storage

# 2. Model ID
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

# 4. Download tokenizer and model (with quantization)
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float32,
    low_cpu_mem_usage=True
)

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [4]:
# 4. Save tokenizer and model to a local directory
model_dir = "model_weights"  # A local directory to save the model and tokenizer
os.makedirs(model_dir, exist_ok=True)

# Save tokenizer
tokenizer.save_pretrained(model_dir)

# Save model weights
model.save_pretrained(model_dir, safe_serialization=True)

In [5]:
# prompt: what is the size of my saved model?

import os

total_size = 0
for dirpath, dirnames, filenames in os.walk(model_dir):
    for f in filenames:
        fp = os.path.join(dirpath, f)
        # skip if it is symbolic link
        if not os.path.islink(fp):
            total_size += os.path.getsize(fp)

print(f"Size of the saved model: {total_size / (1024**3):.2f} GB")

Size of the saved model: 6.63 GB


In [6]:
# 5. Upload files to GCS
def upload_to_gcs(local_dir, gcs_prefix):
    for root, _, files in os.walk(local_dir):
        for filename in files:
            local_filepath = os.path.join(root, filename)
            gcs_path = os.path.join(gcs_prefix, os.path.relpath(local_filepath, local_dir))
            blob = bucket.blob(gcs_path)
            print(f"Uploading {local_filepath} to {gcs_path}...")
            blob.upload_from_filename(local_filepath)

# Upload model weights and tokenizer files
upload_to_gcs(model_dir, "model_weights_deepseek")

print("Model and tokenizer uploaded successfully!")

Uploading model_weights/config.json to model_weights_deepseek/config.json...
Uploading model_weights/tokenizer_config.json to model_weights_deepseek/tokenizer_config.json...
Uploading model_weights/model-00002-of-00002.safetensors to model_weights_deepseek/model-00002-of-00002.safetensors...
Uploading model_weights/generation_config.json to model_weights_deepseek/generation_config.json...
Uploading model_weights/special_tokens_map.json to model_weights_deepseek/special_tokens_map.json...
Uploading model_weights/tokenizer.json to model_weights_deepseek/tokenizer.json...
Uploading model_weights/model.safetensors.index.json to model_weights_deepseek/model.safetensors.index.json...
Uploading model_weights/model-00001-of-00002.safetensors to model_weights_deepseek/model-00001-of-00002.safetensors...
Model and tokenizer uploaded successfully!
